In [ ]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [ ]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [ ]:
arxiv.invoke("Attention is all you need")

In [ ]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=3,doc_content_chars_max=1000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)

In [ ]:
wiki.invoke("What is Generative AI")

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = "ReAct_agent"

In [ ]:
# Custom Function
# Doc string inside the functin is important to write because it help to understand the work of the tool/function by the LLM 
def multiply(a:int, b:int) -> int:
    """ Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a*b

def add(a:int, b:int) -> int:
    """ Add a and b.

    Args:
        a: first int
        b: second int
    """
    return a+b

def divide(a:int, b:int) -> float:
    """ Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a/b
tools = [arxiv,wiki,add,multiply,divide]

In [ ]:
## Tavily seacrh tool
from langchain_tavily import TavilySearch
tavily = TavilySearch(max_result=3,topic='general',include_images=True)

In [ ]:
tavily.invoke("Give me latest news about AI within ongoing month")

In [ ]:
tools = [arxiv,wiki,tavily,add,multiply,divide]

In [ ]:
## Initialize my LLm Model
from langchain_groq import  ChatGroq
llm = ChatGroq(model='llama-3.1-8b-instant')

llm_with_tools = llm.bind_tools(tools)

In [ ]:
from pprint import pprint
from langchain_core.messages import HumanMessage,AIMessage
llm_with_tools.invoke([HumanMessage(content="What is the recent AI news")])

In [ ]:
# State Schema
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from typing import Annotated
class State(TypedDict):
    messages : Annotated[list[AnyMessage],add_messages]



In [ ]:
# Building the graph
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image,display

# node definition
def tool_calling_llm(state:State):
    return {"messages": [llm_with_tools.invoke(state['messages'])]}

#add node
builder = StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

# adding edges
builder.add_edge(START,'tool_calling_llm')
builder.add_conditional_edges("tool_calling_llm",tools_condition)
builder.add_edge('tools',"tool_calling_llm")

graph=builder.compile()

#view
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({"messages":HumanMessage(content="Provide me the top 5 AI recent news from september 10 2025, and add 5 plus 4 and then divide the result by 3")})
for m in messages['messages']:
    m.pretty_print()

### GRaph with Memory

In [ ]:
# Building the graph
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image,display

# node definition
def tool_calling_llm(state:State):
    return {"messages": [llm_with_tools.invoke(state['messages'])]}

#add node
builder = StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

# adding edges
builder.add_edge(START,'tool_calling_llm')
builder.add_conditional_edges("tool_calling_llm",tools_condition)
builder.add_edge('tools',"tool_calling_llm")


In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

graph_memory=builder.compile(checkpointer=memory)

#view
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
## Specify the thread
config = {"configurable": {"thread_id":1}}
# specify the input
messages = [HumanMessage(content="Add 12 and 13")]
messages = graph_memory.invoke({"messages":messages},config=config)
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="Add that number to 25")]
messages = graph_memory.invoke({"messages":messages},config=config)
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="then multiply that number by 2")]
messages = graph_memory.invoke({"messages":messages},config=config)
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="What is 4 plus 5")]
messages = graph_memory.invoke({"messages":messages},config=config)
for m in messages['messages']:
    m.pretty_print()